In [1]:
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

C:\Users\1686110\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
data = ["D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes",
       "D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes",
       "D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes",
       "D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes"]
    

In [3]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [4]:
max_epochs = 200
vec_size = 200
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model_to_check_one")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

In [1]:
import pandas as pd
# from bert_serving.client import BertClient
from elasticsearch import Elasticsearch

In [6]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
model= Doc2Vec.load("d2v.model_to_check_one")

In [8]:
def create_index():
    request_body = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },

        'mappings': {
                'properties': {
                    'ticket': {
                        'type': 'text'
                    },
                    'ticket_vector': {
                            "type": "dense_vector",
                            "dims": 200
                          }
                }}
    }
    print("creating 'qna_index' index...")
    create_index = es.indices.create(index="repeat", body=request_body)
    print("Index-",create_index)

In [9]:
create_index()

creating 'qna_index' index...
Index- {'acknowledged': True, 'shards_acknowledged': True, 'index': 'repeat'}


In [10]:
def Create_Body(total, ticket_vector):
    print("creating body")
    try:
        RecordBody = {
            "ticket": total ,
            "ticket_vector": ticket_vector,
             }

        return RecordBody
    except:
        print("Body for this record cannot be created.")

In [12]:
def Index_Record(RecordBody):
    output = es.index(index='repeat', body=RecordBody)
    if (output['_id']):
        return 1

In [13]:
def insert_into_es():
    print("inside dunc")
    total = ["D&B: Nodes/NEs screen goes blank within 1 min",
            "D&B: Nodes/NEs screen goes blank within 1 min",
            "D&B: Nodes/NEs screen goes blank within 1 min",
            "D&B: Nodes/NEs screen goes blank within 1 min"] 
    for i in total:
        print(i)
        test_data = word_tokenize(i.lower())
        ticket_vector = model.infer_vector(test_data)
        print(ticket_vector)
#         ticket_vector = ticket_vector[0].tolist()
        print(ticket_vector)
        record_body = Create_Body(i, ticket_vector)
        print(record_body)
        count = Index_Record(record_body)
    print("DDDDDDDDDDDDDDDDDDDDDOOOOOOOOOOOOOONNNNEEEEEEEEEEEE")

In [23]:
insert_into_es()

inside dunc
D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes
[-0.16236866 -0.2330661  -0.07037757 -0.01674313  0.03391415  0.0123934
  0.03956462  0.08466798 -0.1533438   0.06233316  0.00855346 -0.06265575
  0.0539742   0.13338387 -0.11736023  0.15006106  0.07317849  0.16645099
 -0.05595331 -0.21969971  0.03232265 -0.06889917  0.17854317  0.04973891
  0.01674536 -0.03821346 -0.02360691  0.02227233 -0.07997401 -0.03635391
  0.13941303  0.07785275  0.04670346 -0.02900426 -0.01373997  0.04272233
 -0.00364297 -0.09845781  0.01053022 -0.04659735  0.00389555  0.02205577
 -0.08064824 -0.05990589 -0.08827291 -0.09806106 -0.02103367 -0.11873781
  0.09303872 -0.06790241 -0.11866132 -0.21680386 -0.12011854 -0.07558794
  0.02428182  0.10056546  0.05619148 -0.07332272 -0.1162471   0.14569241
  0.05316617  0.07534139  0.18063718  0.11453751 -0.1680717   0.2055772
  0.13277471  0.17358741 -0.02089085  0.039652   -0.01574722 -0.00333744
 -0.04034143  0.03420258  0.0316

D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes
[-0.25280774 -0.36343142 -0.11079593 -0.02609832  0.05225817  0.02017328
  0.06199756  0.13102467 -0.23779404  0.09687791  0.01386647 -0.09822872
  0.08396913  0.20709734 -0.1826474   0.23327331  0.11276935  0.25881213
 -0.08715744 -0.34162778  0.04957572 -0.10756489  0.27714482  0.07720285
  0.02586724 -0.05888213 -0.03749933  0.03502011 -0.12471089 -0.05692676
  0.21702585  0.12136115  0.07155713 -0.04459365 -0.0212287   0.06586187
 -0.00523126 -0.15367159  0.01666187 -0.07169316  0.0059513   0.03449224
 -0.12523057 -0.09353726 -0.13607742 -0.15335892 -0.03140004 -0.18547973
  0.14466678 -0.10576805 -0.18432304 -0.33812556 -0.18784495 -0.1166667
  0.03912212  0.15797614  0.08814769 -0.11466496 -0.18184914  0.22772858
  0.08405153  0.11837244  0.2803539   0.17803672 -0.26073357  0.32077566
  0.20770334  0.27097803 -0.03294545  0.06201477 -0.02511036 -0.00447249
 -0.06176514  0.05200119  0.0502751   0.171

In [24]:
text = input("Enter query: ")

x = word_tokenize(text.lower())
# vector = model.infer_vector(i)
embeddings = model.infer_vector(x)
x = embeddings
x = x.tolist()


script_query = {
    "script_score": {
        "query": {"match_all": {}},
        "script": {
            "source": "cosineSimilarity(params.query_vector, doc['ticket_vector']) + 1.0",
            "params": {"query_vector": x}
        }
    }
}


response = es.search(
    index="repeat",
    body={
        "size": 10,
        "query": script_query,
        "_source": {"includes": ["ticket", "body"]}
    }
)

for hit in response["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"])
    print()

Enter query: D&B: Nodes/NEs screen goes blank within 1 min
id: ae9TYHkBE3YZmXGoQa53, score: 0.005934028
{'ticket': 'D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes'}

id: Zu9QYHkBE3YZmXGoG64U, score: 0.00590495
{'ticket': 'D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes'}

id: Ze9QYHkBE3YZmXGoGa7k, score: 0.0058987997
{'ticket': 'D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes'}

id: aO9QYHkBE3YZmXGoG679, score: 0.0058717
{'ticket': 'D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes'}

id: au9TYHkBE3YZmXGoQq6C, score: 0.0058386503
{'ticket': 'D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes'}

id: Z-9QYHkBE3YZmXGoG66Q, score: 0.005827133
{'ticket': 'D&B : NFMT comet events are not received also NFMT UI is not accessible sometimes'}

id: a-9TYHkBE3YZmXGoQq78, score: 0.005819834
{'ticket': 'D&B : NFMT comet events are not r

In [1]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)

HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)')))


OSError: Can't load config for 'bert-base-uncased'. Make sure that:

- 'bert-base-uncased' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'bert-base-uncased' is the correct path to a directory containing a config.json file

